<a href="https://colab.research.google.com/github/veritaem/AB-Demo/blob/master/DS_Unit_2_SPRINT_CHALLENGE_4_Model_Validation_(with_feature_engineering).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Data Science Unit 2 Sprint Challenge 4 — Model Validation

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

## Predicting Blood Donations

Our dataset is from a mobile blood donation vehicle in Taiwan. The Blood Transfusion Service Center drives to different universities and collects blood as part of a blood drive.

The goal is to predict the last column, whether the donor made a donation in March 2007, using information about each donor's history. We'll measure success using recall score as the model evaluation metric.

Good data-driven systems for tracking and predicting donations and supply needs can improve the entire supply chain, making sure that more patients get the blood transfusions they need.

#### Run this cell to load the data:

In [1]:
!pip install category_encoders

In [0]:
import numpy as np
from sklearn.metrics import recall_score 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
import category_encoders as ce

In [0]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')

df = df.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

## Part 1.1 — Begin with baselines

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You don't need to split the data into train and test sets yet. You can answer this question either with a scikit-learn function or with a pandas function.)

In [4]:
df.shape

(748, 5)

In [5]:
df.dtypes

months_since_last_donation     int64
number_of_donations            int64
total_volume_donated           int64
months_since_first_donation    int64
made_donation_in_march_2007    int64
dtype: object

In [6]:
df.isna().sum()

months_since_last_donation     0
number_of_donations            0
total_volume_donated           0
months_since_first_donation    0
made_donation_in_march_2007    0
dtype: int64

In [7]:
df.head()
#ok so the data is numeric and in binary classification form, coolsies!

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [8]:
df.describe()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [9]:
df.made_donation_in_march_2007.mean()

0.23796791443850268

In [10]:
from sklearn.metrics import mean_absolute_error
baseline = [df.made_donation_in_march_2007.mean()] * len(df.made_donation_in_march_2007)  
mean_absolute_error(df.made_donation_in_march_2007, baseline)

0.36267837227258426

What **recall score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of recall.)

In [11]:
maj_classification = df.made_donation_in_march_2007.mode()
y_pred = np.full(shape=df['made_donation_in_march_2007'].shape, fill_value=maj_classification)
recall_score(df['made_donation_in_march_2007'], y_pred)

0.0

##Features to engineer

In [0]:
#bin the months to reduce outliers and normalize 
mo_since_last_bin = [-1, 2, 4, 6, 8, 12, 100]
mo_since_last_label = [1, 2, 3, 4, 5, 6]
df['cat_bin_mo_since_last'] = pd.cut(df['months_since_last_donation'], bins = mo_since_last_bin, labels = mo_since_last_label)

In [0]:
#i figure ppl who give more per visit might also give this time?
df['how_bloody_per_visit'] = df['total_volume_donated'] / df['number_of_donations']

In [0]:
#people who in general give a lot might also give this time?

df['donation_density'] = df['number_of_donations'] / df['months_since_first_donation'] - df['months_since_last_donation']

In [0]:
pf = PolynomialFeatures(2)
pf

In [15]:
df.head()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007,cat_bin_mo_since_last,how_bloody_per_visit,donation_density
0,2,50,12500,98,1,1,250.0,-1.489796
1,0,13,3250,28,1,1,250.0,0.464286
2,1,16,4000,35,1,1,250.0,-0.542857
3,2,20,5000,45,1,1,250.0,-1.555556
4,1,24,6000,77,0,1,250.0,-0.688312


## Part 1.2 — Split data

In this Sprint Challenge, you will use "Cross-Validation with Independent Test Set" for your model evaluation protocol.

First, **split the data into `X_train, X_test, y_train, y_test`**, with random shuffle. (You can include 75% of the data in the train set, and hold out 25% for the test set.)


In [0]:
X = df.drop(['made_donation_in_march_2007'], axis = 1)
y = df['made_donation_in_march_2007']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, train_size = .75, test_size = .25)

## Part 2.1 — Make a pipeline

Make a **pipeline** which includes:
- Preprocessing with any scikit-learn [**Scaler**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)
- Feature selection with **[`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)([`f_classif`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html))**
- Classification with [**`LogisticRegression`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [0]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names = True),
    PolynomialFeatures(2),
    StandardScaler(),
    PowerTransformer(),
    SelectKBest(f_classif),
    LogisticRegression()
)

## Part 2.2 — Do Grid Search Cross-Validation

Do [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) with your pipeline. Use **5 folds** and **recall score**.

Include these **parameters for your grid:**

#### `SelectKBest`
- `k : 1, 2, 3, 4`

#### `LogisticRegression`
- `class_weight : None, 'balanced'`
- `C : .0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0`


**Fit** on the appropriate data.

In [19]:
param_grid = {
    'logisticregression__C': [.0001, .001, .01, .1, 1, 10, 100, 1000, 10000],
    'logisticregression__class_weight':['balanced', None],
    'selectkbest__k': range(1, 5)}


gs = GridSearchCV(pipeline, param_grid=param_grid, cv=5, 
                  scoring='recall', 
                  verbose=0)
gs.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:2778: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 0  6 33] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:2778: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 0  6 

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('onehotencoder', OneHotEncoder(cols=None, drop_invariant=False, handle_unknown='impute',
       impute_missing=True, return_df=True, use_cat_names=True, verbose=0)), ('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('standardscaler', StandardSc...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'logisticregression__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 'logisticregression__class_weight': ['balanced', None], 'selectkbest__k': range(1, 5)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='recall', verbose=0)

## Part 3 — Show best score and parameters

Display your **best cross-validation score**, and the **best parameters** (the values of `k, class_weight, C`) from the grid search.

(You're not evaluated here on how good your score is, or which parameters you find. You're only evaluated on being able to display the information. There are several ways you can get the information, and any way is acceptable.)

In [20]:
validation_score = gs.best_score_

print('Cross-Validation Score:', validation_score)

print('Best estimator:', gs.best_estimator_)

Cross-Validation Score: 0.7620979020979022
Best estimator: Pipeline(memory=None,
     steps=[('onehotencoder', OneHotEncoder(cols=[], drop_invariant=False, handle_unknown='impute',
       impute_missing=True, return_df=True, use_cat_names=True, verbose=0)), ('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('standardscaler', StandardScal...penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False))])


In [32]:
selector = gs.best_estimator_.named_steps['polynomialfeatures']
all_names = X_train.columns
selected_mask = selector.get_feature_names()


print('Features selected:')
for name in selected_mask:
    print(name)

Features selected:
1
x0
x1
x2
x3
x4
x5
x6
x0^2
x0 x1
x0 x2
x0 x3
x0 x4
x0 x5
x0 x6
x1^2
x1 x2
x1 x3
x1 x4
x1 x5
x1 x6
x2^2
x2 x3
x2 x4
x2 x5
x2 x6
x3^2
x3 x4
x3 x5
x3 x6
x4^2
x4 x5
x4 x6
x5^2
x5 x6
x6^2


In [33]:
y_pred = gs.predict(X_test)

test_score = mean_absolute_error(y_test, y_pred)
print('Test Score:', test_score)

Test Score: 0.35294117647058826


## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <th colspan="2" rowspan="2"></th>
    <th colspan="2">Predicted</th>
  </tr>
  <tr>
    <th>Negative</th>
    <th>Positive</th>
  </tr>
  <tr>
    <th rowspan="2">Actual</th>
    <th>Negative</th>
    <td>85</td>
    <td>58</td>
  </tr>
  <tr>
    <th>Positive</th>
    <td>8</td>
    <td>36</td>
  </tr>
</table>

Calculate accuracy

In [0]:
#the number you predicted correctly divided by every one of your predictions
acc = (85 + 36) / (85+36+58+8)
acc

Calculate precision

In [0]:
#number of times you guessed yes and were right divided by your total predicted yeses
prec = 36 / (58 + 36)
prec

Calculate recall

In [0]:
#the number of times you guessed yes and were correct divided by total yeses in the set
recall = 36 / (8+36)
recall

In [0]:
'''F1 Score'''

2 * ((prec * recall) / (prec + recall))

## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Add transformations in your pipeline and parameters in your grid, to try improving your cross-validation score.

### Part 3
Show names of selected features. Then do a final evaluation on the test set — what is the test score?

### Part 4
Calculate F1 score and False Positive Rate. 